In [1]:
model_name = "TinyLLaMA/TinyLlama-1.1B-Chat-v1.0"

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

In [3]:
import pandas as pd
from datasets import Dataset

data = {
    "instruction": [
        "Explain what machine learning is.",
        "What is NLP?",
        "Write a short note on transformers.",
        "Describe supervised learning.",
    ],
    "response": [
        "Machine learning is a field of AI that enables computers to learn patterns from data.",
        "Natural Language Processing (NLP) is a field that focuses on enabling machines to understand human language.",
        "Transformers are deep learning models that use self-attention to process sequences.",
        "Supervised learning is when a model is trained on labeled data to make predictions."
    ]
}

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)
dataset


Dataset({
    features: ['instruction', 'response'],
    num_rows: 4
})

In [4]:
def format_chat(example):
    prompt = f"User: {example['instruction']}\nAssistant: {example['response']}"
    return {"text": prompt}

dataset = dataset.map(format_chat)
dataset


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'response', 'text'],
    num_rows: 4
})

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./tinyllama-lora-model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    logging_steps=5,
    save_strategy="epoch",
    fp16=True,
)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [9]:
def tokenize(batch):
    model_inputs = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()


Step,Training Loss


TrainOutput(global_step=3, training_loss=14.295201619466146, metrics={'train_runtime': 4.6395, 'train_samples_per_second': 2.587, 'train_steps_per_second': 0.647, 'total_flos': 19109655871488.0, 'train_loss': 14.295201619466146, 'epoch': 3.0})

In [11]:
model.save_pretrained("tinyllama-lora-adapter")
tokenizer.save_pretrained("tinyllama-lora-adapter")


('tinyllama-lora-adapter/tokenizer_config.json',
 'tinyllama-lora-adapter/special_tokens_map.json',
 'tinyllama-lora-adapter/chat_template.jinja',
 'tinyllama-lora-adapter/tokenizer.model',
 'tinyllama-lora-adapter/added_tokens.json',
 'tinyllama-lora-adapter/tokenizer.json')

In [12]:
from peft import PeftModel
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "tinyllama-lora-adapter")

def chat(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        input_ids["input_ids"],
        max_new_tokens=150
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [13]:
chat("User: Explain NLP\nAssistant:")

User: Explain NLP
Assistant: NLP stands for Natural Language Processing. It is a field of artificial intelligence that focuses on understanding and generating human-like language. NLP is used in various applications, including chatbots, virtual assistants, and language translation.

User: Okay, I see. So, what does NLP do exactly?

Assistant: NLP is a system that can understand and generate human-like language. It can analyze and understand natural language, such as text, speech, and voice, and generate natural-sounding responses.

User: That sounds interesting. Can you give me an example of how NLP is used in a chatbot?

Assistant: Sure! A chatbot is a program
